In [0]:
from pprint import pprint as pp

# Chapter 3: Machine Learning Pipelines

### specify database and create dataframes

In [0]:
%sql
use u23_2_3_schema;
show tables;

database,tableName,isTemporary
u23_2_3_schema,airports,false
u23_2_3_schema,flights,false
u23_2_3_schema,planes,false


In [0]:
print(spark.catalog.listTables())

[Table(name='airports', database='u23_2_3_schema', description=None, tableType='MANAGED', isTemporary=False), Table(name='flights', database='u23_2_3_schema', description=None, tableType='MANAGED', isTemporary=False), Table(name='planes', database='u23_2_3_schema', description=None, tableType='MANAGED', isTemporary=False)]

In [0]:
flights = spark.table('flights')
print('length = ', flights.count())
flights.show(5)

length = 10000
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
2014| 12| 8| 658| -7| 935| -5| VX| N846VA| 1780| SEA| LAX| 132| 954| 6| 58|
2014| 1| 22| 1040| 5| 1505| 5| AS| N559AS| 851| SEA| HNL| 360| 2677| 10| 40|
2014| 3| 9| 1443| -2| 1652| 2| VX| N847VA| 755| SEA| SFO| 111| 679| 14| 43|
2014| 4| 9| 1705| 45| 1839| 34| WN| N360SW| 344| PDX| SJC| 83| 569| 17| 5|
2014| 3| 9| 754| -1| 1015| 1| AS| N612AS| 522| SEA| BUR| 127| 937| 7| 54|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
only showing top 5 rows

In [0]:
airports = spark.table('airports')
print('length = ', airports.count())
airports.show(5)

length = 1397
+---+--------------------+----------+-----------+----+---+---+
faa| name| lat| lon| alt| tz|dst|
+---+--------------------+----------+-----------+----+---+---+
04G| Lansdowne Airport|41.1304722|-80.6195833|1044| -5| A|
06A|Moton Field Munic...|32.4605722|-85.6800278| 264| -5| A|
06C| Schaumburg Regional|41.9893408|-88.1012428| 801| -6| A|
06N| Randall Airport| 41.431912|-74.3915611| 523| -5| A|
09J|Jekyll Island Air...|31.0744722|-81.4277778| 11| -4| A|
+---+--------------------+----------+-----------+----+---+---+
only showing top 5 rows

In [0]:
planes = spark.table('planes')
print('length = ', planes.count())
planes.show(5)

length = 2628
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
tailnum|year| type| manufacturer| model|engines|seats|speed| engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
 N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214| 2| 182| NA|Turbo-fan|
 N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214| 2| 182| NA|Turbo-fan|
 N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214| 2| 182| NA|Turbo-fan|
 N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214| 2| 182| NA|Turbo-fan|
 N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214| 2| 182| NA|Turbo-fan|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
only showing top 5 rows

### join `flights` and `planes`
##### both have `year` and `tailnum` columns
##### rename `year` column in `planes`

In [0]:
planes = planes.withColumnRenamed('year', 'plane_year')
planes.show(5)

+-------+----------+--------------------+----------------+--------+-------+-----+-----+---------+
tailnum|plane_year| type| manufacturer| model|engines|seats|speed| engine|
+-------+----------+--------------------+----------------+--------+-------+-----+-----+---------+
 N102UW| 1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214| 2| 182| NA|Turbo-fan|
 N103US| 1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214| 2| 182| NA|Turbo-fan|
 N104UW| 1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214| 2| 182| NA|Turbo-fan|
 N105UW| 1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214| 2| 182| NA|Turbo-fan|
 N107US| 1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214| 2| 182| NA|Turbo-fan|
+-------+----------+--------------------+----------------+--------+-------+-----+-----+---------+
only showing top 5 rows

In [0]:
model_data = flights.join(planes, on='tailnum', how="leftouter")
model_data.schema
model_data.show(5)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+
tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year| type|manufacturer| model|engines|seats|speed| engine|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+
 N846VA|2014| 12| 8| 658| -7| 935| -5| VX| 1780| SEA| LAX| 132| 954| 6| 58| 2011|Fixed wing multi ...| AIRBUS|A320-214| 2| 182| NA|Turbo-fan|
 N559AS|2014| 1| 22| 1040| 5| 1505| 5| AS| 851| SEA| HNL| 360| 2677| 10| 40| 2006|Fixed wing multi ...| BOEING| 737-890| 2| 149| NA|Turbo-fan|
 N847VA|2014| 3| 9| 1443| -2| 1652| 2| VX| 755| SEA| SFO| 111| 679| 14| 43| 2011|Fixed wing multi ...| AIRBUS|A320-214| 2| 182| NA|Turbo-fan|
 N360SW|2014| 4| 9| 1705| 45| 1839| 34| WN| 344| PDX| SJC| 83| 569| 17| 5| 1992|Fixed wing multi ...| BOEING| 737-3H4| 2| 149| NA|Turbo-fan|
 N612AS|2014| 3| 9| 754| -1| 1015| 1| AS| 522| SEA| BUR| 127| 937| 7| 54| 1999|Fixed wing multi ...| BOEING| 737-790| 2| 151| NA|Turbo-jet|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+
only showing top 5 rows

### change type of some columns

In [0]:
model_data = model_data.withColumn('arr_delay', model_data.arr_delay.cast('integer'))
model_data.printSchema()

root
-- tailnum: string (nullable = true)
-- year: integer (nullable = true)
-- month: integer (nullable = true)
-- day: integer (nullable = true)
-- dep_time: integer (nullable = true)
-- dep_delay: integer (nullable = true)
-- arr_time: integer (nullable = true)
-- arr_delay: integer (nullable = true)
-- carrier: string (nullable = true)
-- flight: integer (nullable = true)
-- origin: string (nullable = true)
-- dest: string (nullable = true)
-- air_time: integer (nullable = true)
-- distance: integer (nullable = true)
-- hour: integer (nullable = true)
-- minute: integer (nullable = true)
-- plane_year: integer (nullable = true)
-- type: string (nullable = true)
-- manufacturer: string (nullable = true)
-- model: string (nullable = true)
-- engines: integer (nullable = true)
-- seats: integer (nullable = true)
-- speed: string (nullable = true)
-- engine: string (nullable = true)

### add column for plane age

In [0]:
model_data = model_data.withColumn('plane_age', model_data.year - model_data.plane_year)
model_data.show(5)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+
tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year| type|manufacturer| model|engines|seats|speed| engine|plane_age|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+
 N846VA|2014| 12| 8| 658| -7| 935| -5| VX| 1780| SEA| LAX| 132| 954| 6| 58| 2011|Fixed wing multi ...| AIRBUS|A320-214| 2| 182| NA|Turbo-fan| 3|
 N559AS|2014| 1| 22| 1040| 5| 1505| 5| AS| 851| SEA| HNL| 360| 2677| 10| 40| 2006|Fixed wing multi ...| BOEING| 737-890| 2| 149| NA|Turbo-fan| 8|
 N847VA|2014| 3| 9| 1443| -2| 1652| 2| VX| 755| SEA| SFO| 111| 679| 14| 43| 2011|Fixed wing multi ...| AIRBUS|A320-214| 2| 182| NA|Turbo-fan| 3|
 N360SW|2014| 4| 9| 1705| 45| 1839| 34| WN| 344| PDX| SJC| 83| 569| 17| 5| 1992|Fixed wing multi ...| BOEING| 737-3H4| 2| 149| NA|Turbo-fan| 22|
 N612AS|2014| 3| 9| 754| -1| 1015| 1| AS| 522| SEA| BUR| 127| 937| 7| 54| 1999|Fixed wing multi ...| BOEING| 737-790| 2| 151| NA|Turbo-jet| 15|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+
only showing top 5 rows

### add `label` column indicating whether or not flight was late

In [0]:
model_data = model_data.withColumn('is_late', model_data.arr_delay > 0)
model_data = model_data.withColumn('label', model_data.is_late.cast('integer'))
model_data.printSchema()
model_data.show(5)

root
-- tailnum: string (nullable = true)
-- year: integer (nullable = true)
-- month: integer (nullable = true)
-- day: integer (nullable = true)
-- dep_time: integer (nullable = true)
-- dep_delay: integer (nullable = true)
-- arr_time: integer (nullable = true)
-- arr_delay: integer (nullable = true)
-- carrier: string (nullable = true)
-- flight: integer (nullable = true)
-- origin: string (nullable = true)
-- dest: string (nullable = true)
-- air_time: integer (nullable = true)
-- distance: integer (nullable = true)
-- hour: integer (nullable = true)
-- minute: integer (nullable = true)
-- plane_year: integer (nullable = true)
-- type: string (nullable = true)
-- manufacturer: string (nullable = true)
-- model: string (nullable = true)
-- engines: integer (nullable = true)
-- seats: integer (nullable = true)
-- speed: string (nullable = true)
-- engine: string (nullable = true)
-- plane_age: integer (nullable = true)
-- is_late: boolean (nullable = true)
-- label: integer (nullable = true)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+-------+-----+
tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year| type|manufacturer| model|engines|seats|speed| engine|plane_age|is_late|label|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+-------+-----+
 N846VA|2014| 12| 8| 658| -7| 935| -5| VX| 1780| SEA| LAX| 132| 954| 6| 58| 2011|Fixed wing multi ...| AIRBUS|A320-214| 2| 182| NA|Turbo-fan| 3| false| 0|
 N559AS|2014| 1| 22| 1040| 5| 1505| 5| AS| 851| SEA| HNL| 360| 2677| 10| 40| 2006|Fixed wing multi ...| BOEING| 737-890| 2| 149| NA|Turbo-fan| 8| true| 1|
 N847VA|2014| 3| 9| 1443| -2| 1652| 2| VX| 755| SEA| SFO| 111| 679| 14| 43| 2011|Fixed wing multi ...| AIRBUS|A320-214| 2| 182| NA|Turbo-fan| 3| true| 1|
 N360SW|2014| 4| 9| 1705| 45| 1839| 34| WN| 344| PDX| SJC| 83| 569| 17| 5| 1992|Fixed wing multi ...| BOEING| 737-3H4| 2| 149| NA|Turbo-fan| 22| true| 1|
 N612AS|2014| 3| 9| 754| -1| 1015| 1| AS| 522| SEA| BUR| 127| 937| 7| 54| 1999|Fixed wing multi ...| BOEING| 737-790| 2| 151| NA|Turbo-jet| 15| true| 1|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+-------+-----+
only showing top 5 rows

### remove rows with missing values

In [0]:
print('num rows original = ', model_data.count())

num rows original = 10000

In [0]:
temp_data = model_data.where(model_data.arr_delay.isNotNull() & 
                             model_data.dep_delay.isNotNull() &
                             model_data.air_time.isNotNull() &
                             model_data.plane_year.isNotNull()
                            )
print('num rows filtered = ', temp_data.count())

num rows filtered = 9303

In [0]:
model_data = model_data.where(model_data.arr_delay.isNotNull() & 
                             model_data.dep_delay.isNotNull() &
                             model_data.air_time.isNotNull() &
                             model_data.plane_year.isNotNull()
                            )
print('num rows filtered = ', model_data.count())

num rows filtered = 9303

### `import` for pre-processing

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

### one-hot encoding of `carrier`

In [0]:
carr_indexer = StringIndexer(inputCol='carrier', outputCol='carrier_index')
carr_encoder = OneHotEncoder(inputCol='carrier_index', outputCol='carrier_fact')

### one-hot encoding of `dest`

In [0]:
dest_indexer = StringIndexer(inputCol='dest', outputCol='dest_index')
dest_encoder = OneHotEncoder(inputCol='dest_index', outputCol='dest_fact')

### assemble data

In [0]:
vec_assembler = VectorAssembler(inputCols=['month', 'air_time', 'carrier_fact', 'dest_fact', 'plane_age'],
                                outputCol='features')

### create pipeline

In [0]:
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler])

### execute pipeline

In [0]:
piped_data = flights_pipe.fit(model_data).transform(model_data)

In [0]:
print(type(piped_data))
print('length piped_data = ', piped_data.count())
piped_data.printSchema()
piped_data.show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
length piped_data = 9303
root
-- tailnum: string (nullable = true)
-- year: integer (nullable = true)
-- month: integer (nullable = true)
-- day: integer (nullable = true)
-- dep_time: integer (nullable = true)
-- dep_delay: integer (nullable = true)
-- arr_time: integer (nullable = true)
-- arr_delay: integer (nullable = true)
-- carrier: string (nullable = true)
-- flight: integer (nullable = true)
-- origin: string (nullable = true)
-- dest: string (nullable = true)
-- air_time: integer (nullable = true)
-- distance: integer (nullable = true)
-- hour: integer (nullable = true)
-- minute: integer (nullable = true)
-- plane_year: integer (nullable = true)
-- type: string (nullable = true)
-- manufacturer: string (nullable = true)
-- model: string (nullable = true)
-- engines: integer (nullable = true)
-- seats: integer (nullable = true)
-- speed: string (nullable = true)
-- engine: string (nullable = true)
-- plane_age: integer (nullable = true)
-- is_late: boolean (nullable = true)
-- label: integer (nullable = true)
-- dest_index: double (nullable = false)
-- dest_fact: vector (nullable = true)
-- carrier_index: double (nullable = false)
-- carrier_fact: vector (nullable = true)
-- features: vector (nullable = true)

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+-------+-----+----------+---------------+-------------+--------------+--------------------+
tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|plane_year| type|manufacturer| model|engines|seats|speed| engine|plane_age|is_late|label|dest_index| dest_fact|carrier_index| carrier_fact| features|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+-------+-----+----------+---------------+-------------+--------------+--------------------+
 N846VA|2014| 12| 8| 658| -7| 935| -5| VX| 1780| SEA| LAX| 132| 954| 6| 58| 2011|Fixed wing multi ...| AIRBUS|A320-214| 2| 182| NA|Turbo-fan| 3| false| 0| 1.0| (68,[1],[1.0])| 7.0|(10,[7],[1.0])|(81,[0,1,9,13,80]...|
 N559AS|2014| 1| 22| 1040| 5| 1505| 5| AS| 851| SEA| HNL| 360| 2677| 10| 40| 2006|Fixed wing multi ...| BOEING| 737-890| 2| 149| NA|Turbo-fan| 8| true| 1| 19.0|(68,[19],[1.0])| 0.0|(10,[0],[1.0])|(81,[0,1,2,31,80]...|
 N847VA|2014| 3| 9| 1443| -2| 1652| 2| VX| 755| SEA| SFO| 111| 679| 14| 43| 2011|Fixed wing multi ...| AIRBUS|A320-214| 2| 182| NA|Turbo-fan| 3| true| 1| 0.0| (68,[0],[1.0])| 7.0|(10,[7],[1.0])|(81,[0,1,9,12,80]...|
 N360SW|2014| 4| 9| 1705| 45| 1839| 34| WN| 344| PDX| SJC| 83| 569| 17| 5| 1992|Fixed wing multi ...| BOEING| 737-3H4| 2| 149| NA|Turbo-fan| 22| true| 1| 7.0| (68,[7],[1.0])| 1.0|(10,[1],[1.0])|(81,[0,1,3,19,80]...|
 N612AS|2014| 3| 9| 754| -1| 1015| 1| AS| 522| SEA| BUR| 127| 937| 7| 54| 1999|Fixed wing multi ...| BOEING| 737-790| 2| 151| NA|Turbo-jet| 15| true| 1| 22.0|(68,[22],[1.0])| 0.0|(10,[0],[1.0])|(81,[0,1,2,34,80]...|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+----------+--------------------+------------+--------+-------+-----+-----+---------+---------+-------+-----+----------+---------------+-------------+--------------+--------------------+
only showing top 5 rows

### train-test split (60% train)

In [0]:
training, test = piped_data.randomSplit([0.6, 0.4])

In [0]:
print('training set info:')
print(type(training))
print('length = ', training.count())
training.printSchema()

print('test set info:')
print(type(test))
print('length = ', test.count())
test.printSchema()

training set info:
<class 'pyspark.sql.dataframe.DataFrame'>
length = 5579
root
-- tailnum: string (nullable = true)
-- year: integer (nullable = true)
-- month: integer (nullable = true)
-- day: integer (nullable = true)
-- dep_time: integer (nullable = true)
-- dep_delay: integer (nullable = true)
-- arr_time: integer (nullable = true)
-- arr_delay: integer (nullable = true)
-- carrier: string (nullable = true)
-- flight: integer (nullable = true)
-- origin: string (nullable = true)
-- dest: string (nullable = true)
-- air_time: integer (nullable = true)
-- distance: integer (nullable = true)
-- hour: integer (nullable = true)
-- minute: integer (nullable = true)
-- plane_year: integer (nullable = true)
-- type: string (nullable = true)
-- manufacturer: string (nullable = true)
-- model: string (nullable = true)
-- engines: integer (nullable = true)
-- seats: integer (nullable = true)
-- speed: string (nullable = true)
-- engine: string (nullable = true)
-- plane_age: integer (nullable = true)
-- is_late: boolean (nullable = true)
-- label: integer (nullable = true)
-- dest_index: double (nullable = false)
-- dest_fact: vector (nullable = true)
-- carrier_index: double (nullable = false)
-- carrier_fact: vector (nullable = true)
-- features: vector (nullable = true)

test set info:
<class 'pyspark.sql.dataframe.DataFrame'>
length = 3724
root
-- tailnum: string (nullable = true)
-- year: integer (nullable = true)
-- month: integer (nullable = true)
-- day: integer (nullable = true)
-- dep_time: integer (nullable = true)
-- dep_delay: integer (nullable = true)
-- arr_time: integer (nullable = true)
-- arr_delay: integer (nullable = true)
-- carrier: string (nullable = true)
-- flight: integer (nullable = true)
-- origin: string (nullable = true)
-- dest: string (nullable = true)
-- air_time: integer (nullable = true)
-- distance: integer (nullable = true)
-- hour: integer (nullable = true)
-- minute: integer (nullable = true)
-- plane_year: integer (nullable = true)
-- type: string (nullable = true)
-- manufacturer: string (nullable = true)
-- model: string (nullable = true)
-- engines: integer (nullable = true)
-- seats: integer (nullable = true)
-- speed: string (nullable = true)
-- engine: string (nullable = true)
-- plane_age: integer (nullable = true)
-- is_late: boolean (nullable = true)
-- label: integer (nullable = true)
-- dest_index: double (nullable = false)
-- dest_fact: vector (nullable = true)
-- carrier_index: double (nullable = false)
-- carrier_fact: vector (nullable = true)
-- features: vector (nullable = true)

# Chapter 4: Model Tuning and Selection

### `import` for modeling

In [0]:
import numpy as np

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

### instantiate objects

##### model and evaluator

In [0]:
lr = LogisticRegression()
evaluator = BinaryClassificationEvaluator(metricName = 'areaUnderROC')

##### hyper-parameter grid

In [0]:
grid = ParamGridBuilder()
grid = grid.addGrid(lr.regParam,        np.arange(0, 0.1, 0.01))
grid = grid.addGrid(lr.elasticNetParam, [0, 1])
grid = grid.build()

##### cross validator

In [0]:
cv = CrossValidator(estimator=lr,
                    estimatorParamMaps=grid,
                    evaluator=evaluator)

### perform modeling

##### fit

In [0]:
models = cv.fit(training)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [0]:
print(type(models))
print(models)

<class 'pyspark.ml.tuning.CrossValidatorModel'>
CrossValidatorModel_a49ec5c8dc60

In [0]:
best_lr = models.bestModel
print(type(best_lr))
print(best_lr)

<class 'pyspark.ml.classification.LogisticRegressionModel'>
LogisticRegressionModel: uid=LogisticRegression_7a5c418ce136, numClasses=2, numFeatures=81

##### investigate `models`

In [0]:
pp(models.avgMetrics)
print('length = ', len(models.avgMetrics))

[0.688991160268508,
 0.688991160268508,
 0.60795776234943,
 0.5653798139560634,
 0.5863638157866604,
 0.5631604905550756,
 0.5772100734328848,
 0.5356587471938142,
 0.5720856369956288,
 0.5345831133192068,
 0.5688544454842643,
 0.5,
 0.5667010782467635,
 0.5,
 0.5650136957450332,
 0.5,
 0.5636517802383961,
 0.5,
 0.5626035412021596,
 0.5]
length = 20

In [0]:
print(models.estimatorParamMaps)

CrossValidatorModel_a49ec5c8dc60__estimatorParamMaps

In [0]:
pp(models.explainParams())

('estimator: estimator to be cross-validated (current: '
 'LogisticRegression_7a5c418ce136)\n'
 'estimatorParamMaps: estimator param maps (current: '
 "[{Param(parent='LogisticRegression_7a5c418ce136', name='regParam', "
 "doc='regularization parameter (>= 0).'): 0.0, "
 "Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', "
 "doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the "
 "penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}, "
 "{Param(parent='LogisticRegression_7a5c418ce136', name='regParam', "
 "doc='regularization parameter (>= 0).'): 0.0, "
 "Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', "
 "doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the "
 "penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0}, "
 "{Param(parent='LogisticRegression_7a5c418ce136', name='regParam', "
 "doc='regularization parameter (>= 0).'): 0.01, "
 "Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', "
 "doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the "
 "penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}, "
 "{Param(parent='LogisticRegression_7a5c418ce136', name='regParam', "
 "doc='regularization parameter (>= 0).'): 0.01, "
 "Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', "
 "doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the "
 "penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0}, "
 "{Param(parent='LogisticRegression_7a5c418ce136', name='regParam', "
 "doc='regularization parameter (>= 0).'): 0.02, "
 "Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', "
 "doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the "
 "penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}, "
 "{Param(parent='LogisticRegression_7a5c418ce136', name='regParam', "
 "doc='regularization parameter (>= 0).'): 0.02, "
 "Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', "
 "doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the "
 "penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0}, "
 "{Param(parent='LogisticRegression_7a5c418ce136', name='regParam', "
 "doc='regularization parameter (>= 0).'): 0.03, "
 "Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', "
 "doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the "
 "penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}, "
 "{Param(parent='LogisticRegression_7a5c418ce136', name='regParam', "
 "doc='regularization parameter (>= 0).'): 0.03, "
 "Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', "
 "doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the "
 "penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0}, "
 "{Param(parent='LogisticRegression_7a5c418ce136', name='regParam', "
 "doc='regularization parameter (>= 0).'): 0.04, "
 "Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', "
 "doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the "
 "penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}, "
 "{Param(parent='LogisticRegression_7a5c418ce136', name='regParam', "
 "doc='regularization parameter (>= 0).'): 0.04, "
 "Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', "
 "doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the "
 "penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0}, "
 "{Param(parent='LogisticRegression_7a5c418ce136', name='regParam', "
 "doc='regularization parameter (>= 0).'): 0.05, "
 "Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', "
 "doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the "
 "penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0}, "
 

In [0]:
pp(models.getEstimatorParamMaps())

[{Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.0},
 {Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.0},
 {Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.01},
 {Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.01},
 {Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.02},
 {Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.02},
 {Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.03},
 {Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.03},
 {Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.04},
 {Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.04},
 {Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.05},
 {Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1.0,
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'): 0.05},
 {Param(parent='Lo

##### investigate best model

In [0]:
print('elasticNetParam = ', best_lr.getElasticNetParam()) 
print('regParam        = ', best_lr.getRegParam())

elasticNetParam = 0.0
regParam = 0.0

In [0]:
pp(best_lr.summary)

<pyspark.ml.classification.BinaryLogisticRegressionTrainingSummary object at 0x7ffa24695d10>

In [0]:
pp(best_lr.params)

[Param(parent='LogisticRegression_7a5c418ce136', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'),
 Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'),
 Param(parent='LogisticRegression_7a5c418ce136', name='featuresCol', doc='features column name.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='fitIntercept', doc='whether to fit an intercept term.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='labelCol', doc='label column name.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='lowerBoundsOnCoefficients', doc='The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='lowerBoundsOnIntercepts', doc='The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal with 1 for binomial regression, or the number oflasses for multinomial regression.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='maxIter', doc='max number of iterations (>= 0).'),
 Param(parent='LogisticRegression_7a5c418ce136', name='predictionCol', doc='prediction column name.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='probabilityCol', doc='Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='rawPredictionCol', doc='raw prediction (a.k.a. confidence) column name.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='regParam', doc='regularization parameter (>= 0).'),
 Param(parent='LogisticRegression_7a5c418ce136', name='standardization', doc='whether to standardize the training features before fitting the model.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='threshold', doc='Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p].'),
 Param(parent='LogisticRegression_7a5c418ce136', name='thresholds', doc="Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold."),
 Param(parent='LogisticRegression_7a5c418ce136', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'),
 Param(parent='LogisticRegression_7a5c418ce136', name='upperBoundsOnCoefficients', doc='The upper bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='upperBoundsOnIntercepts', doc='The upper bounds on intercepts if fitting under bound constrained optimization. The bound vector size must be equal with 1 for binomial regression, or the number of classes for multinomial regression.'),
 Param(parent='LogisticRegression_7a5c418ce136', name='weightCol', doc='weight column name. If this is not set or empty, we treat all instance weights as 1.0.')]

In [0]:
pp(best_lr.getParam('elasticNetParam'))

Param(parent='LogisticRegression_7a5c418ce136', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.')

##### use best model with test data
`elasticNetParam = 0`, `regParam = 0`; these are default values

In [0]:
test_results = best_lr.transform(test)
print(evaluator.evaluate(test_results))

0.6945255751352096